<a href="https://colab.research.google.com/github/lmontaldo/hts/blob/main/Brasil_hts_Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip uninstall statsforecast

In [2]:
!pip install statsforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.8/279.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 5.1 MB/s eta 0:00:00
  Created wheel for fugue-sql-antlr: filename=fugue_sql_antlr-0.2.0-py3-none-any.whl size=158196 sha256=f1a43c662f35e10d64bedc87e68b763372cdf56a39ed1207f1885d4ef3191482
  Stored in directory: /root/.cache/pip/wheels/5a/b5/4e/216953a1c711da55de29ed7ecf158b4a5bf32ef93d69ad66dd
Successfully built fugue-sql-antlr


In [3]:
from sklearn.model_selection import train_test_split
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import collections
import statistics

In [4]:
!pip show statsforecast

Name: statsforecast
Version: 1.7.1
Summary: Time series forecasting suite using statistical models
Home-page: https://github.com/Nixtla/statsforecast/
Author: Nixtla
Author-email: business@nixtla.io
License: Apache Software License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: cloudpickle, fugue, numba, numpy, pandas, scipy, statsmodels, tqdm, utilsforecast
Required-by: 


In [5]:
#from datasetsforecast.hierarchical import HierarchicalData
#obtain hierarchical reconciliation methods and evaluation
#from hierarchicalforecast.core import HierarchicalReconciliation
#from hierarchicalforecast.evaluation import HierarchicalEvaluation
#from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut
# compute base forecast no coherent
from statsforecast.core import StatsForecast

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:
#from statsforecast.models import auto_arima, naive

In [7]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
file_path = '/content/drive/My Drive/hts/br.csv'


In [9]:
df = pd.read_csv(file_path, index_col=0)

In [10]:
#from hierarchicalforecast.utils import aggregate

In [11]:
df.head()

,window,supermarket,local,local_name,shipping_method,max_capacity,percentage_consume,day_week,y
ds,,,,,,,,,
2023-09-02,10:00 - 12:00,Gbarbosa,B006,Jardins,Retirada en Tienda,3,100.0,5,3
2023-09-02,10:00 - 12:00,Gbarbosa,B015,Feira de Santana (Rodoviária),Entrega Agendada,5,100.0,5,5
2023-09-02,10:00 - 12:00,Gbarbosa,B015,Feira de Santana (Rodoviária),Retirada en Tienda,3,100.0,5,3
2023-09-02,10:00 - 12:00,Gbarbosa,B035,Costa Azul,Retirada en Tienda,5,100.0,5,5
2023-09-02,10:00 - 12:00,Gbarbosa,B084,Stella Maris,Entrega Agendada,5,60.0,5,3


In [12]:
#file_path = 'df_limpio.csv'
#df = pd.read_csv(file_path, index_col=0)

## Análisis de los datos

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46667 entries, 2023-09-02 to 2023-12-30
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   window              46667 non-null  object 
 1   supermarket         46667 non-null  object 
 2   local               46667 non-null  object 
 3   local_name          46667 non-null  object 
 4   shipping_method     46667 non-null  object 
 5   max_capacity        46667 non-null  int64  
 6   percentage_consume  46667 non-null  float64
 7   day_week            46667 non-null  int64  
 8   y                   46667 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 3.6+ MB


In [14]:
df.columns

Index(['window', 'supermarket', 'local', 'local_name', 'shipping_method',
       'max_capacity', 'percentage_consume', 'day_week', 'y'],
      dtype='object')

In [15]:
df.head()

,window,supermarket,local,local_name,shipping_method,max_capacity,percentage_consume,day_week,y
ds,,,,,,,,,
2023-09-02,10:00 - 12:00,Gbarbosa,B006,Jardins,Retirada en Tienda,3,100.0,5,3
2023-09-02,10:00 - 12:00,Gbarbosa,B015,Feira de Santana (Rodoviária),Entrega Agendada,5,100.0,5,5
2023-09-02,10:00 - 12:00,Gbarbosa,B015,Feira de Santana (Rodoviária),Retirada en Tienda,3,100.0,5,3
2023-09-02,10:00 - 12:00,Gbarbosa,B035,Costa Azul,Retirada en Tienda,5,100.0,5,5
2023-09-02,10:00 - 12:00,Gbarbosa,B084,Stella Maris,Entrega Agendada,5,60.0,5,3


### Comenzamos con un único super: Bretas

In [16]:
bretas=df[df.supermarket=="Bretas"]

In [17]:
bretas.shape

(10502, 9)

In [18]:
bretas.columns

Index(['window', 'supermarket', 'local', 'local_name', 'shipping_method',
       'max_capacity', 'percentage_consume', 'day_week', 'y'],
      dtype='object')

In [19]:
bretas = bretas.drop(['supermarket','local_name', 'max_capacity', 'percentage_consume'], axis=1)
bretas.isnull().sum()

window             0
local              0
shipping_method    0
day_week           0
y                  0
dtype: int64

In [20]:
start_date_train=bretas.index.min()
end_date_train='2023-11-30'

In [21]:
train = bretas.loc[(bretas.index >= start_date_train) & (bretas.index <= end_date_train)]
valid=bretas.loc[bretas.index > end_date_train]

# Lista de columnas para establecer la jerarquía

In [22]:
spec = [['local'],
        ['local', 'window'],
        ['local', 'window', 'shipping_method']]

In [23]:
train.head()

,window,local,shipping_method,day_week,y
ds,,,,,
2023-09-02,12:00 - 15:00,B502,Entrega Agendada,5,5
2023-09-02,12:00 - 15:00,B514,Entrega Agendada,5,1
2023-09-02,12:00 - 15:00,B515,Entrega Agendada,5,9
2023-09-02,12:00 - 15:00,B515,Retirada en Tienda,5,5
2023-09-02,12:00 - 15:00,B533,Entrega Agendada,5,6


In [24]:
!pip install hierarchicalforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.8/452.8 kB 13.7 MB/s eta 0:00:00


In [25]:
from hierarchicalforecast.utils import aggregate
train_agg, S_train, tags = aggregate(train, spec)
valid_agg, _, _ = aggregate(valid, spec)

In [26]:
train_agg.sample(5)

,ds,y
unique_id,,
B639/12:00 - 15:00/Entrega Agendada,2023-11-25,6
B514/18:00 - 21:00/Retirada en Tienda,2023-11-01,0
B533/18:00 - 21:00/Entrega Agendada,2023-10-18,2
B534/15:00 - 18:00/Retirada en Tienda,2023-10-29,0
B533/12:00 - 15:00,2023-09-13,1


In [27]:
train_agg_sind=train_agg.reset_index()

In [28]:
train_agg_sind.head()

,unique_id,ds,y
0,B502,2023-09-02,8
1,B502,2023-09-04,2
2,B502,2023-09-05,3
3,B502,2023-09-06,6
4,B502,2023-09-07,1


In [57]:
from statsforecast import StatsForecast
from statsforecast.models import HoltWinters, AutoARIMA, AutoETS, WindowAverage

StatsForecast(models=[HoltWinters,AutoARIMA,WindowAverage])

In [63]:
model = StatsForecast(models=[HoltWinters(season_length=7, error_type='A'),
AutoARIMA(max_p=7, max_q=4, max_P=4, max_Q=4 ) ],
freq='D', n_jobs=-1)
model.fit(train_agg_sind)

StatsForecast(models=[HoltWinters,AutoARIMA])

In [64]:
h=train_agg.index.nunique()

In [65]:
p = model.forecast(h=h, fitted=True)

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:399: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:417: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [60]:
p = model.forecast(h=h)

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:417: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [66]:
p_fitted = model.forecast_fitted_values()

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:417: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [67]:
p_fitted.sample(20)

,ds,y,HoltWinters,AutoARIMA
unique_id,,,,
B533/18:00 - 21:00,2023-10-12,5.0,4.454259,3.880257
B612/12:00 - 15:00/Entrega Agendada,2023-11-11,2.0,0.407746,0.847550
B542/18:00 - 21:00/Barrio Elisson Prieto,2023-10-25,0.0,-0.000905,0.000000
B634/12:00 - 15:00/Retirada en Tienda,2023-10-16,1.0,0.403924,0.000000
B534/9:00 - 12:00/Entrega Agendada,2023-11-08,1.0,2.143295,3.780172
B634/9:00 - 12:00,2023-11-25,1.0,0.153765,0.246003
B634/9:00 - 12:00/Entrega Agendada,2023-11-18,1.0,0.559453,0.210613
B502/12:00 - 15:00,2023-10-03,1.0,1.834444,1.204077
B615/18:00 - 21:00/Entrega Agendada,2023-11-30,6.0,3.487833,3.632617


## Reconciliación

In [68]:
from hierarchicalforecast.methods import BottomUp, TopDown, MinTrace, ERM, OptimalCombination
from hierarchicalforecast.core import HierarchicalReconciliation

In [69]:
reconcilers = [BottomUp(),
               ERM(method='closed'),
               ERM(method='reg'),
               ERM(method='reg_bu')
              ]

In [70]:
rec_model = HierarchicalReconciliation(reconcilers=reconcilers)
p_rec = rec_model.reconcile(Y_hat_df=p, Y_df=p_fitted, S=S_train, tags=tags)

In [71]:
p_rec.columns

Index(['ds', 'HoltWinters', 'AutoARIMA', 'HoltWinters/BottomUp',
       'AutoARIMA/BottomUp', 'HoltWinters/ERM_method-closed_lambda_reg-0.01',
       'AutoARIMA/ERM_method-closed_lambda_reg-0.01',
       'HoltWinters/ERM_method-reg_lambda_reg-0.01',
       'AutoARIMA/ERM_method-reg_lambda_reg-0.01',
       'HoltWinters/ERM_method-reg_bu_lambda_reg-0.01',
       'AutoARIMA/ERM_method-reg_bu_lambda_reg-0.01'],
      dtype='object')

In [36]:
p_rec.sample(10)

,ds,HoltWinters,HoltWinters/BottomUp,HoltWinters/ERM_method-closed_lambda_reg-0.01,HoltWinters/ERM_method-reg_lambda_reg-0.01,HoltWinters/ERM_method-reg_bu_lambda_reg-0.01
unique_id,,,,,,
B534/18:00 - 21:00,2024-04-28,1.277269,0.618138,-9.929620e+00,-10.651327,-9.487297
B615/9:00 - 12:00/Retirada en Tienda,2024-04-03,0.949839,0.949839,2.512810e+00,1.832386,1.375978
B612/15:00 - 18:00/Retirada en Tienda,2024-02-26,0.094912,0.094912,2.418890e-08,0.000000,0.084426
B534/9:00 - 12:00/Entrega Agendada,2024-04-27,1.711453,1.711453,-1.580774e+00,0.776892,0.847844
B639/12:00 - 15:00/Retirada en Tienda,2023-12-26,0.086202,0.086202,-5.288962e-07,0.000000,-0.289888
B615/15:00 - 18:00,2024-02-28,0.581679,0.867388,6.099678e+00,1.443055,-6.426387
B515/15:00 - 18:00,2024-01-16,5.035514,5.929672,4.972826e+00,5.268527,3.245944
B634/18:00 - 21:00/Retirada en Tienda,2024-01-24,0.243361,0.243361,-2.147403e-07,0.000000,-0.079319
B625,2024-01-06,2.061850,3.623801,3.018426e+00,3.868630,-0.107162


In [37]:
p_rec.head()

,ds,HoltWinters,HoltWinters/BottomUp,HoltWinters/ERM_method-closed_lambda_reg-0.01,HoltWinters/ERM_method-reg_lambda_reg-0.01,HoltWinters/ERM_method-reg_bu_lambda_reg-0.01
unique_id,,,,,,
B502,2023-12-01,4.238457,2.246459,7.410601,7.360006,5.644818
B502,2023-12-02,5.071499,4.426829,5.634111,5.806332,6.320116
B502,2023-12-03,0.050524,5.308036,-2.431955,-4.627353,0.841337
B502,2023-12-04,2.901487,2.140838,8.657472,5.822065,4.327937
B502,2023-12-05,4.188846,3.049876,8.287741,7.911525,6.604082


In [72]:
p_rec_sind=p_rec.reset_index()

In [73]:
p_rec.info()

<class 'pandas.core.frame.DataFrame'>
CategoricalIndex: 28224 entries, B502 to B639/9:00 - 12:00/Retirada en Tienda
Data columns (total 11 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   ds                                             28224 non-null  datetime64[ns]
 1   HoltWinters                                    28224 non-null  float32       
 2   AutoARIMA                                      28224 non-null  float32       
 3   HoltWinters/BottomUp                           28224 non-null  float32       
 4   AutoARIMA/BottomUp                             28224 non-null  float32       
 5   HoltWinters/ERM_method-closed_lambda_reg-0.01  28224 non-null  float32       
 6   AutoARIMA/ERM_method-closed_lambda_reg-0.01    28224 non-null  float32       
 7   HoltWinters/ERM_method-reg_lambda_reg-0.01     28224 non-null  float32       
 8   AutoARIMA/ERM_me

In [74]:
valid_agg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4679 entries, B502 to B639/9:00 - 12:00/Retirada en Tienda
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      4679 non-null   datetime64[ns]
 1   y       4679 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 238.7+ KB


In [75]:
valid_agg['ds'] = pd.to_datetime(valid_agg['ds'])

In [76]:
p_rec_ = p_rec.merge(valid_agg, on=['ds', 'unique_id'], how='left')
p_rec_['y'] = p_rec_['y'].fillna(0)
p_rec_ = p_rec_.reset_index(drop=False)

from sklearn.metrics import mean_squared_error

rmse = dict()
for model_ in p_rec.columns[1:]:
    rmse_ = mean_squared_error(p_rec_['y'].values, p_rec_[model_].values, squared=False)/1e3
    # get only the model name
    model__ = model_.split('/')[-1]
    rmse[model__] = rmse_
pd.DataFrame(rmse, index=['RMSE']).T.sort_values('RMSE')

,RMSE
AutoARIMA,0.003639
BottomUp,0.003920
ERM_method-closed_lambda_reg-0.01,0.004567
HoltWinters,0.006174
ERM_method-reg_bu_lambda_reg-0.01,0.006226
ERM_method-reg_lambda_reg-0.01,0.006973


In [ ]:
!pip freeze > requirements.txt

In [ ]:
from google.colab import files

# Download the file
files.download('requirements.txt')
